In [1]:
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd

2023-03-01 03:59:06.037081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 03:59:06.174267: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-01 03:59:06.178853: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 03:59:06.178874: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
model = gensim.models.Word2Vec.load('../lm-vol/2_28_NTML_Pikachu_500_epochs_word2vec.model')

In [3]:
wv = KeyedVectors.load('../lm-vol/2_28_NTML_Pikachu_500_epochs_word2vec.wordvectors', mmap='r')

In [13]:
G = nx.read_graphml('../lm-vol/LANL_test_NTLM_Pikachu_filter_v2.graphml')
train_G = nx.read_graphml('../lm-vol/LANL_train_NTLM_Pikachu_filter_v2.graphml')

In [17]:
test_nodes = set(G.nodes)
train_nodes = set(train_G.nodes)
missing_nodes = test_nodes.difference(train_nodes)
print(len(missing_nodes))
print(len(test_nodes))
print(len(train_nodes))

2027
17621
19540


In [5]:
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]

** Sampled 17847 positive and 17847 negative edges. **


KeyError: "Key 'U5284' not present"

In [ ]:
clf = LogisticRegressionCV(cv=5, max_iter=1000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)

In [ ]:
import pickle
# Save to file in the current working directory
pkl_filename = "../lm-vol/2_18_100_epochs_lr.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [18]:
import pickle
import pandas as pd
with open('../lm-vol/LANL_train_NTLM_Pikachu_filter_v2_lr.pkl', 'rb') as file:
    clf = pickle.load(file)

In [20]:
graph_structure = ('src_user', 'source computer', 'dest_user')
client_to_ip_pred = []
ip_to_service_pred = []
inconclusive = 0
for chunk in pd.read_csv('../lm-vol/LANL_test_NTLM_Pikachu_filter_v2.csv', chunksize=1000000):
#     display(chunk)
    for index, row in chunk.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    chunk['client_to_ip_pred'] = client_to_ip_pred
    chunk['ip_to_service_pred'] = ip_to_service_pred
    break

FileNotFoundError: [Errno 2] No such file or directory: '../lm-vol/LANL_test_NTLM_Pikachu_filter_v2.csv'

In [ ]:
chunk

In [ ]:
anomalous_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)])
normal_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) > 0.1) & (chunk["client_to_ip_pred"].astype(float) > 0.1)])
print(anomalous_edges/(normal_edges+anomalous_edges))
print(anomalous_edges)
print(normal_edges)

In [ ]:
anomalies = pd.read_csv('../lm-vol/LANL_anomalies.csv', low_memory=False)

In [ ]:
anomalies.shape

In [ ]:
4770524/196999999

In [ ]:
anomalies

In [ ]:
red_team = pd.read_csv('../lm-vol/redteam_complete.csv')

In [ ]:
red_team

In [ ]:
true_positive = 0
for a_index, a_row in anomalies.iterrows():
    a_time = int(a_row[0])
    a_source = a_row[1]
    a_destination = a_row[2]
    a_ip = a_row[3]
    for r_index, r_row in red_team.iterrows():
        if r_row['time'] == a_time and r_row['source'] == a_source and r_row['destiantion'] == a_destination and r_row['source_computer'] == a_ip:
            true_positive += 1
            print(a_time)
            break
        elif r_row['time'] > a_time:
            break

In [ ]:
experiment_names = ['2_18']

def operator_hadamard(u, v):
        return u * v

for name in experiment_names:
    with open(f'../lm-vol/{name}_lr.pkl', 'rb') as file:
        clf = pickle.load(file)
    model = gensim.models.Word2Vec.load(f'../lm-vol/{name}_word2vec.model')
    wv = KeyedVectors.load(f'../lm-vol/{name}_word2vec.wordvectors', mmap='r')
    graph_structure = ('source', 'source_computer', 'destination')
    inconclusive = 0
    n = 0
    red = pd.read_csv('../lm-vol/redteam_complete.csv')
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in red.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    red['client_to_ip_pred'] = client_to_ip_pred
    red['ip_to_service_pred'] = ip_to_service_pred
    red_anomalies = red[(red["ip_to_service_pred"].astype(float) <= 0.1) | (red["client_to_ip_pred"].astype(float) <= 0.1)]
    red_invalids = red[(red["ip_to_service_pred"] == None) & (red["client_to_ip_pred"] == None)]
    display(red_anomalies)

    test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
    test = []
    # max = 100
    for index, edge in enumerate(test_G.edges):
    #     if index > max:
    #         break
        try:
            src = wv[edge[0]]
            dest = wv[edge[1]]
            src_to_dest = [operator_hadamard(src, dest)]
            prob = clf.predict_proba(src_to_dest)
            test.append(prob[0][1])
        except:
            test.append(None)
    anomalies = 0
    for prob in test:
        if prob is not None and prob <= 0.1:
            anomalies+=1
    print(anomalies)
#     graph_structure = (1, 3, 2)
#     client_to_ip_pred = []
#     ip_to_service_pred = []
#     inconclusive = 0
#     for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     #     display(chunk)
#         for index, row in chunk.iterrows():
#             try: 
#                 client_embedding = wv[(row[graph_structure[0]])]
#                 ip_embedding = wv[(row[graph_structure[1]])]
#                 service_embedding = wv[(row[graph_structure[2]])]
#                 client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
#                 ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
#                 client_to_ip = clf.predict_proba(client_to_ip_embedding)
#                 ip_to_service = clf.predict_proba(ip_to_service_embedding)
#                 client_to_ip_pred.append(client_to_ip[0][1])
#                 ip_to_service_pred.append(ip_to_service[0][1])
#             except: 
#                 client_to_ip_pred.append(None)
#                 ip_to_service_pred.append(None)
#                 inconclusive += 1
#         chunk['client_to_ip_pred'] = client_to_ip_pred
#         chunk['ip_to_service_pred'] = ip_to_service_pred
#         chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
#         chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
#         display(chunk_anomalies)
#         break
    break